# Kernel Semantico

In questo esempio di codice, utilizzerai il framework AI [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) per creare un agente di base.

L'obiettivo di questo esempio è mostrarti i passaggi che utilizzeremo successivamente negli ulteriori esempi di codice per implementare i diversi modelli agentici.


## Importa i Pacchetti Python Necessari


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Creazione del Client

In questo esempio, utilizzeremo [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) per accedere all'LLM.

L'`ai_model_id` è definito come `gpt-4o-mini`. Prova a cambiare il modello con un altro disponibile nel marketplace di GitHub Models per vedere risultati diversi.

Per utilizzare l'`Azure Inference SDK`, che viene usato per il `base_url` di GitHub Models, utilizzeremo il connettore `OpenAIChatCompletion` all'interno di Semantic Kernel. Esistono anche altri [connettori disponibili](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) per utilizzare Semantic Kernel con altri fornitori di modelli.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creazione dell'Agente

Di seguito creiamo l'Agente chiamato `TravelAgent`.

Per questo esempio, stiamo utilizzando istruzioni molto semplici. Puoi modificare queste istruzioni per vedere come l'agente risponde in modo diverso.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Esecuzione dell'Agente

Ora possiamo eseguire l'Agente definendo un thread di tipo `ChatHistoryAgentThread`. Eventuali messaggi di sistema richiesti vengono forniti come argomento keyword `messages` alla funzione invoke_stream dell'agente.

Una volta definiti questi, creiamo un `user_inputs` che rappresenta ciò che l'utente invia all'agente. In questo caso, abbiamo impostato questo messaggio su `Plan me a sunny vacation`.

Sentiti libero di modificare questo messaggio per vedere come l'agente risponde in modo diverso.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche potrebbero contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si consiglia una traduzione professionale eseguita da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
